### 資料參考來源：
https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_sql.html

In [24]:
import pandas as pd

url = "https://raw.github.com/pandas-dev/pandas/master/pandas/tests/io/data/csv/tips.csv"
tips = pd.read_csv(url)
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


##### 大部分的pandas 都會copy Series/Dataframe

In [22]:
# 排序
tips.sort_values("tip")                                         #由小到大
tips.sort_values("tip", ascending=False)                        #由大到小

#GroupBy 
tips.groupby("sex").size()                                      #size() 只有取得這個欄位
tips.groupby("sex").count()                                     #全部欄位的統計
tips.groupby("sex")["total_bill"].count()                       #針對total_bill欄位的統計

import numpy as np
# 加入npmpy數學統計套件
# SELECT day, AVG(tip), COUNT(*)            FROM tips GROUP BY day;
tips.groupby("day").agg({"tip": np.mean, "day": np.size})

# SELECT smoker, day, COUNT(*), AVG(tip)    FROM tips GROUP BY smoker, day;
tips.groupby(["smoker", "day"]).agg({"tip": [np.size, np.mean]})


tip          
             size      mean
smoker day                 
No     Fri    4.0  2.812500
       Sat   45.0  3.102889
       Sun   57.0  3.167895
       Thur  45.0  2.673778
Yes    Fri   15.0  2.714000
       Sat   42.0  2.875476
       Sun   19.0  3.516842
       Thur  17.0  3.030000

### Join


In [11]:

import pandas as pd
import numpy as np
df1 = pd.DataFrame({"key": ["A", "B", "C", "D"], "value": np.random.randn(4)})
df2 = pd.DataFrame({"key": ["B", "D", "D", "E"], "value": np.random.randn(4)})

df1,df2


(  key     value
 0   A  0.069774
 1   B -1.052237
 2   C -0.219424
 3   D -1.271718,
   key     value
 0   B  0.655577
 1   D  0.375253
 2   D  0.175897
 3   E -0.417229)

### UNION

In [20]:
df1 = pd.DataFrame(
    {"city": ["Chicago", "San Francisco", "New York City"], "rank": range(1, 4)}
)
df2 = pd.DataFrame(
    {"city": ["Chicago", "Boston", "Los Angeles"], "rank": [1, 4, 5]}
)
df1,df2


(            city  rank
 0        Chicago     1
 1  San Francisco     2
 2  New York City     3,
           city  rank
 0      Chicago     1
 1       Boston     4
 2  Los Angeles     5)

In [22]:
pd.concat([df1, df2])                       # SQL’s UNION is similar to UNION ALL, however UNION will remove duplicate rows.
pd.concat([df1, df2]).drop_duplicates()     # concat().drop_duplicates() 移除資料有重複的部分。


,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
1,Boston,4
2,Los Angeles,5


### pandas equivalents for some SQL analytic and aggregate functions
Top n rows with offset

In [25]:
tips.nlargest(10 + 5, columns="tip").tail(10)

,total_bill,tip,sex,smoker,day,time,size
183,23.17,6.50,Male,Yes,Sun,Dinner,4
214,28.17,6.50,Female,Yes,Sat,Dinner,3
47,32.40,6.00,Male,No,Sun,Dinner,4
239,29.03,5.92,Male,No,Sat,Dinner,3
88,24.71,5.85,Male,No,Thur,Lunch,2
181,23.33,5.65,Male,Yes,Sun,Dinner,2
44,30.40,5.60,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
85,34.83,5.17,Female,No,Thur,Lunch,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4


-- Oracle's ROW_NUMBER() analytic function
-- 前三天的最大的total_bill金額統計?

    SELECT * FROM (
      SELECT
      t.*,
      ROW_NUMBER() OVER(PARTITION BY day ORDER BY total_bill DESC) AS rn
    FROM tips t
  )
     WHERE rn < 3
     ORDER BY day, rn;

In [33]:
# tips.assign(
#     rn=tips.sort_values(["total_bill"], ascending=False)
#     .groupby(["day"])
#     .cumcount()
#     + 1
# )

(
    tips.assign(
        rn=tips.sort_values(["total_bill"], ascending=False)
        .groupby(["day"])
        .cumcount()
        + 1
    )
    .query("rn < 3")
    .sort_values(["day", "rn"])
)

,total_bill,tip,sex,smoker,day,time,size,rn
95,40.17,4.73,Male,Yes,Fri,Dinner,4,1
90,28.97,3.00,Male,Yes,Fri,Dinner,2,2
170,50.81,10.00,Male,Yes,Sat,Dinner,3,1
212,48.33,9.00,Male,No,Sat,Dinner,4,2
156,48.17,5.00,Male,No,Sun,Dinner,6,1
182,45.35,3.50,Male,Yes,Sun,Dinner,3,2
197,43.11,5.00,Female,Yes,Thur,Lunch,4,1
142,41.19,5.00,Male,No,Thur,Lunch,5,2


In [ ]:
# UPDATE tips
# SET tip = tip*2
# WHERE tip < 2;

tips.loc[tips["tip"] < 2, "tip"] *= 2

# DELETE FROM tips WHERE tip > 9;
# In pandas we select the rows that should remain instead of deleting them:
tips = tips.loc[tips["tip"] <= 9]